<a href="https://colab.research.google.com/github/thoppae/Agent/blob/main/A1_PPL_Cricket_Launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import streamlit as st
import csv
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# --- The Fix: Apply nest_asyncio patch ---
import nest_asyncio

nest_asyncio.apply()
# --- End of Fix ---

# --- 1. Configuration and Setup ---
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

st.set_page_config(page_title="Palkar Premier League Q&A", layout="wide")
st.title("Palkar Premier League Q&A Bot 🏏")

if not API_KEY:
    st.error("🚨 Google API Key not found. Please set the GOOGLE_API_KEY in your .env file.")
    st.stop()


# --- 2. Core LangChain and Model Initialization ---
@st.cache_resource
def get_models():
    """Initializes and returns the LLM and embeddings model."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        google_api_key=API_KEY,
        temperature=0.2,
        convert_system_message_to_human=True
    )
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=API_KEY
    )
    return llm, embeddings


llm, embeddings = get_models()


# --- Helper function for cleaning CSV data ---
def _clean_csv_dict(row: dict) -> dict:
    """
    Cleans the keys of a dictionary read from a CSV.
    It strips whitespace and quote characters from each key.
    """
    cleaned_dict = {}
    for key, value in row.items():
        if key:  # Ensure key is not None
            cleaned_key = key.strip().strip('"')
            cleaned_dict[cleaned_key] = value
    return cleaned_dict


# --- 3. Data Loading and Vector Database Creation (for Q&A) ---
@st.cache_resource
def create_combined_vectordb(data_path: str, rules_path: str, _embeddings):
    """
    Loads data for the RAG Q&A system. It creates a main summary document for each player,
    plus additional, focused "expert" documents for important attributes like committee
    roles and injuries to improve retrieval accuracy.
    """
    player_docs = []
    with open(data_path, mode='r', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            cleaned_row = _clean_csv_dict(row)
            player_name = cleaned_row.get("Player Name", "Unknown Player")

            # --- Create the main, comprehensive document for each player ---
            main_content = ""
            for key, value in cleaned_row.items():
                if value and value.strip():
                    main_content += f"{key}: {value.strip()}. "

            if main_content:
                metadata = {"source": data_path, "row": i + 2, "content_type": "Player Summary"}
                player_docs.append(Document(page_content=main_content.strip(), metadata=metadata))

            # --- IMPROVEMENT: Create additional, focused documents for key attributes ---

            # Create a dedicated document for Committee information
            committee = cleaned_row.get("Committee")
            committee_task = cleaned_row.get("Committee Task")
            if committee and committee.strip():
                committee_content = f"Regarding player {player_name}: They are on the {committee} committee. The task is described as: {committee_task}"
                metadata = {"source": data_path, "row": i + 2, "content_type": "Committee Role"}
                player_docs.append(Document(page_content=committee_content, metadata=metadata))

            # Create a dedicated document for Injury information
            injury_status = cleaned_row.get("Injury")
            if injury_status and injury_status.strip().lower() == 'yes':
                injury_type = cleaned_row.get("Injury Type", "not specified")
                injury_desc = cleaned_row.get("Injury Description", "no description")
                injury_content = f"Regarding player {player_name}: They have an injury. Status: {injury_status}. Type: {injury_type}. Description: {injury_desc}."
                metadata = {"source": data_path, "row": i + 2, "content_type": "Injury Status"}
                player_docs.append(Document(page_content=injury_content, metadata=metadata))

    # --- Process Rule Data (as before) ---
    rules_docs = []
    with open(rules_path, mode='r', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            rule_text = row.get('Team Selection Rules')
            if rule_text:
                metadata = {"source": rules_path, "row": i + 2, "content_type": "Rule"}
                rules_docs.append(Document(page_content=rule_text, metadata=metadata))

    all_docs = player_docs + rules_docs
    st.info(f"Creating vector database for Q&A from {len(all_docs)} total documents (including focused experts)...")
    vectordb = FAISS.from_documents(documents=all_docs, embedding=_embeddings)
    st.success("Q&A Vector database created successfully!")
    return vectordb


vectordb = create_combined_vectordb(
    data_path='data.csv',
    rules_path='rules.csv',
    _embeddings=embeddings
)


# --- 4. System Creation (Q&A Chain and Team Formation Helpers) ---
def get_qa_chain(db, _llm):
    """Builds the simple RetrievalQA chain for factual questions."""
    prompt_template = """
    You are an assistant for the Palkar Premier League.
    Use the following pieces of context to answer the question at the end.
    If the answer is not found in the context, state "I don't have enough information to answer that."
    CONTEXT: {context}
    QUESTION: {question}
    HELPFUL ANSWER:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = RetrievalQA.from_chain_type(
        llm=_llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 8}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return chain


def get_full_context_from_files(data_path: str, rules_path: str) -> (str, str):
    """Reads the raw CSV data and formats it as a string for the LLM."""
    with open(rules_path, mode='r', encoding='utf-8-sig') as f:
        rules_text = f.read()

    players_text = ""
    with open(data_path, mode='r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            cleaned_row = _clean_csv_dict(row)
            players_text += str(cleaned_row) + "\n"

    return rules_text, players_text


qa_chain = get_qa_chain(vectordb, llm)

# --- 5. Streamlit User Interface ---
st.subheader("Ask a Factual Question")
st.write("Use this for simple questions about players or rules. For example: *'Who are the wicket keepers?'*")

with st.form("qa_form"):
    question = st.text_input("Your Question:", key="question_input")
    submit_qa_button = st.form_submit_button("Ask")

if submit_qa_button and question:
    with st.spinner("Searching for an answer..."):
        try:
            response = qa_chain.invoke(question)
            st.write(response['result'])
            with st.expander("Show Sources"):
                st.write("The answer was generated from the following sources:")
                for doc in response['source_documents']:
                    st.info(
                        f"Source: {doc.metadata.get('source', 'N/A')}, Row: {doc.metadata.get('row', 'N/A')}, Type: {doc.metadata.get('content_type', 'N/A')}")
                    st.text(doc.page_content)
        except Exception as e:
            st.error(f"An error occurred: {e}")

st.divider()

st.subheader("Form Balanced Teams (Advanced Reasoning)")
st.write("Use this to ask the AI to perform the complex task of creating teams based on all rules and players.")

num_teams = st.number_input("How many teams to form?", min_value=1, max_value=10, value=1, step=1)

if st.button("Generate Teams"):
    with st.spinner(f"Performing complex reasoning to form {num_teams} teams. This may take a moment..."):
        try:
            rules_context, players_context = get_full_context_from_files(
                data_path='data.csv',
                rules_path='rules.csv'
            )

            team_formation_prompt = f"""
            You are a world-class expert in cricket team selection and logical reasoning. Your task is to create {num_teams} balanced teams from a list of players, strictly following a given set of rules.

            **HERE ARE THE TEAM SELECTION RULES (You MUST follow all of them):**
            ---
            {rules_context}
            ---

            **HERE IS THE COMPLETE LIST OF AVAILABLE PLAYERS (in dictionary format with cleaned keys):**
            ---
            {players_context}
            ---

            **YOUR TASK:**
            1.  **Analyze all rules and players thoroughly.**
            2.  **Create exactly {num_teams} teams.** Distribute the players as evenly as possible.
            3.  **Ensure every team composition satisfies ALL the rules.** For example, check the wicket keeper count, spinner types, injury limits, etc., for EACH team.
            4.  **Provide a step-by-step thought process** explaining how you are forming the teams and verifying the rules.
            5.  **Finally, present the final teams clearly.** List the player names for each team.

            Begin your reasoning now.
            """

            response = llm.invoke(team_formation_prompt)
            st.write(response.content)

        except Exception as e:
            st.error(f"An error occurred during team formation: {e}")